In [1]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

In [2]:
# lyrics and labels
df = pd.read_csv('train.csv')
lyrics = df['Lyric']
labels = df['Label']
val_df = pd.read_csv('validate.csv')
val_lyrics = df['Lyric']
val_labels = df['Label']

# parameter setting
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'


In [3]:
#Train
# Tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(lyrics)
word_index = tokenizer.word_index

# sequence and fill
sequences = tokenizer.texts_to_sequences(lyrics)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

labels = np.array(labels)-1



In [4]:
#Validate
# Tokenizer
val_tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
val_tokenizer.fit_on_texts(val_lyrics)

# sequence and fill
val_sequences = val_tokenizer.texts_to_sequences(val_lyrics)
val_padded_sequences = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_labels = np.array(val_labels)-1

In [5]:
# Construct the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(GRU(64))
model.add(Dense(10, activation='softmax')) # Because our emotional labels are 1-10, the last layer uses 10 neurons

# Build an optimizer to set the LR
optimizer = Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])   #loss function


In [ ]:
# Model fit
model.fit(padded_sequences, labels, epochs=100,validation_data=(val_padded_sequences, val_labels),batch_size=32)

In [ ]:
# Evaluate model performance
loss, accuracy = model.evaluate(padded_sequences, labels)
print(f'Model accuracy: {accuracy*100:.2f}%')

In [8]:
df2 = pd.read_csv('test.csv')
df2['Lyric'] = df2['Lyric'].fillna('')  # 
new_lyrics = df2['Lyric']
new_labels = df2['Label']

# Tokenizer
tokenizer2 = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer2.fit_on_texts(new_lyrics)

# sequence and fill
sequences = tokenizer2.texts_to_sequences(new_lyrics)
new_padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [ ]:
# predict the test
y_pred = model.predict(new_padded_sequences)

# Take the label with the highest probability as the prediction result
y_pred = np.argmax(y_pred, axis=1) + 1

# Accuracy
accuracy = np.sum(y_pred == new_labels) / len(new_labels)
print('Accuracy: ', accuracy)
